# Data Wrangling

In [1]:
#datalink because github doesn't l https://www.kaggle.com/competitions/store-sales-time-series-forecasting/data

In [2]:
import pandas as pd
import numpy as np

In [3]:
#load and check data
data_path = "/Users/carlriemann/Documents/data/"
train_file = data_path + "train.csv"
test_file = data_path + "test.csv"
stores_file = data_path + "stores.csv"
oil_file = data_path + "oil.csv"
holidays_file = data_path + "holidays_events.csv"

train_df = pd.read_csv(train_file, parse_dates=['date'])
test_df = pd.read_csv(test_file, parse_dates=['date'])
stores_df = pd.read_csv(stores_file)
oil_df = pd.read_csv(oil_file, parse_dates=['date'])
holidays_df = pd.read_csv(holidays_file, parse_dates=['date'])


print("Train DataFrame:")
print(train_df.head(), '\n')


print("Test DataFrame:")
print(test_df.head(), '\n')

      
print("Stores DataFrame:")
print(stores_df.head(), '\n')

print("Oil DataFrame:")
print(oil_df.head(), '\n')

print("Holidays DataFrame:")
print(holidays_df.head(), '\n')

Train DataFrame:
   id       date  store_nbr      family  sales  onpromotion
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0
1   1 2013-01-01          1   BABY CARE    0.0            0
2   2 2013-01-01          1      BEAUTY    0.0            0
3   3 2013-01-01          1   BEVERAGES    0.0            0
4   4 2013-01-01          1       BOOKS    0.0            0 

Test DataFrame:
        id       date  store_nbr      family  onpromotion
0  3000888 2017-08-16          1  AUTOMOTIVE            0
1  3000889 2017-08-16          1   BABY CARE            0
2  3000890 2017-08-16          1      BEAUTY            2
3  3000891 2017-08-16          1   BEVERAGES           20
4  3000892 2017-08-16          1       BOOKS            0 

Stores DataFrame:
   store_nbr           city                           state type  cluster
0          1          Quito                       Pichincha    D       13
1          2          Quito                       Pichincha    D       13
2          3   

In [4]:
#check for missing values
print("Missing values in Train DataFrame:\n", train_df.isnull().sum(), '\n')
print("Missing values in Test DataFrame:\n", test_df.isnull().sum(), '\n')
print("Missing values in Stores DataFrame:\n", stores_df.isnull().sum(), '\n')
print("Missing values in Oil DataFrame:\n", oil_df.isnull().sum(), '\n')
print("Missing values in Holidays DataFrame:\n", holidays_df.isnull().sum(), '\n')

Missing values in Train DataFrame:
 id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64 

Missing values in Test DataFrame:
 id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64 

Missing values in Stores DataFrame:
 store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64 

Missing values in Oil DataFrame:
 date           0
dcoilwtico    43
dtype: int64 

Missing values in Holidays DataFrame:
 date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64 



In [5]:
#fill "dcoilwtico" by linear interpolation, and forwardfill/backwardfill
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].interpolate(method='linear')
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].ffill().bfill()

In [6]:
#fix holidays_df
transfer_holidays = holidays_df[holidays_df['type'] == 'Transfer']

#For each transfer holiday, find the corresponding original holiday where 'transferred' is True
for index, row in transfer_holidays.iterrows():
    #Find the original holiday row where transferred is True and the names match (looking for the word Traslado)
    original_holiday = holidays_df[(holidays_df['description'] == row['description'].replace("Traslado ", "")) & (holidays_df['transferred'] == True)]
    
    if not original_holiday.empty:
        #update the row where 'holiday_type' is 'Transfer' with info from the original holiday
        holidays_df.at[index, 'description'] = original_holiday['description'].values[0]
        holidays_df.at[index, 'date'] = row['date']  #keep the 'Transfer' holiday date
        holidays_df.at[index, 'type'] = 'TransferredHoliday'

# Step 3: Drop the 'transferred' column
holidays_df.drop(columns=['transferred'], inplace=True)

In [7]:
print("Missing values in Oil DataFrame:\n", oil_df.isnull().sum(), '\n')

Missing values in Oil DataFrame:
 date          0
dcoilwtico    0
dtype: int64 



In [8]:
#creating new date features columns
train_df['year'] = train_df['date'].dt.year
train_df['month'] = train_df['date'].dt.month
train_df['day'] = train_df['date'].dt.day
train_df['day_of_week'] = train_df['date'].dt.dayofweek
train_df['week_of_year'] = train_df['date'].dt.isocalendar().week

test_df['year'] = test_df['date'].dt.year
test_df['month'] = test_df['date'].dt.month
test_df['day'] = test_df['date'].dt.day
test_df['day_of_week'] = test_df['date'].dt.dayofweek
test_df['week_of_year'] = test_df['date'].dt.isocalendar().week

In [9]:
#Creating lag features for sales to capture recent trends.
train_df['lag_1'] = train_df.groupby(['store_nbr', 'family'])['sales'].shift(1)
train_df['lag_7'] = train_df.groupby(['store_nbr', 'family'])['sales'].shift(7)

#Create rolling averages, help the model detect smoother trends
train_df['rolling_mean_7'] = train_df.groupby(['store_nbr', 'family'], group_keys=False)['sales'] \
    .rolling(window=7).mean().reset_index(level=['store_nbr', 'family'], drop=True)

train_df['rolling_mean_30'] = train_df.groupby(['store_nbr', 'family'], group_keys=False)['sales'] \
    .rolling(window=30).mean().reset_index(level=['store_nbr', 'family'], drop=True)

#check the results
print(train_df[['sales', 'lag_1', 'lag_7', 'rolling_mean_7', 'rolling_mean_30']].head(10))

   sales  lag_1  lag_7  rolling_mean_7  rolling_mean_30
0    0.0    NaN    NaN             NaN              NaN
1    0.0    NaN    NaN             NaN              NaN
2    0.0    NaN    NaN             NaN              NaN
3    0.0    NaN    NaN             NaN              NaN
4    0.0    NaN    NaN             NaN              NaN
5    0.0    NaN    NaN             NaN              NaN
6    0.0    NaN    NaN             NaN              NaN
7    0.0    NaN    NaN             NaN              NaN
8    0.0    NaN    NaN             NaN              NaN
9    0.0    NaN    NaN             NaN              NaN


#REMEMBER NAN VALUES ARE ADDED TO DATES WITH NO past INFO

In [10]:
train_df['lag_1'] = train_df['lag_1'].ffill()
train_df['lag_7'] = train_df['lag_7'].ffill()
train_df['rolling_mean_7'] = train_df['rolling_mean_7'].ffill()
train_df['rolling_mean_30'] = train_df['rolling_mean_30'].ffill()
train_df['lag_1'] = train_df['lag_1'].ffill()
train_df['lag_7'] = train_df['lag_7'].ffill()
train_df['rolling_mean_7'] = train_df['rolling_mean_7'].ffill()
train_df['rolling_mean_30'] = train_df['rolling_mean_30'].ffill()

In [11]:
# Creating lag features for the test set
test_df['lag_1'] = test_df.groupby(['store_nbr', 'family'])['onpromotion'].shift(1)
test_df['lag_7'] = test_df.groupby(['store_nbr', 'family'])['onpromotion'].shift(7)

# Creating rolling statistics for the test set
test_df['rolling_mean_7'] = test_df.groupby(['store_nbr', 'family'], group_keys=False)['onpromotion'] \
    .rolling(window=7).mean().reset_index(level=['store_nbr', 'family'], drop=True)
test_df['rolling_mean_30'] = test_df.groupby(['store_nbr', 'family'], group_keys=False)['onpromotion'] \
    .rolling(window=30).mean().reset_index(level=['store_nbr', 'family'], drop=True)

# Forward fill any NaN values in the lag and rolling features for the test set
test_df['lag_1'] = test_df['lag_1'].ffill()
test_df['lag_7'] = test_df['lag_7'].ffill()
test_df['rolling_mean_7'] = test_df['rolling_mean_7'].ffill()
test_df['rolling_mean_30'] = test_df['rolling_mean_30'].ffill()
test_df['lag_1'] = test_df['lag_1'].bfill()
test_df['lag_7'] = test_df['lag_7'].bfill()
test_df['rolling_mean_7'] = test_df['rolling_mean_7'].bfill()
test_df['rolling_mean_30'] = test_df['rolling_mean_30'].bfill()

In [12]:
print("Columns in train_df:", train_df.columns)
print("Columns in test_df:", test_df.columns)
print("Columns in holidays_df:", holidays_df.columns)
print("Columns in oil_df:", oil_df.columns)
print("Columns in stores_df:", stores_df.columns)

Columns in train_df: Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion', 'year',
       'month', 'day', 'day_of_week', 'week_of_year', 'lag_1', 'lag_7',
       'rolling_mean_7', 'rolling_mean_30'],
      dtype='object')
Columns in test_df: Index(['id', 'date', 'store_nbr', 'family', 'onpromotion', 'year', 'month',
       'day', 'day_of_week', 'week_of_year', 'lag_1', 'lag_7',
       'rolling_mean_7', 'rolling_mean_30'],
      dtype='object')
Columns in holidays_df: Index(['date', 'type', 'locale', 'locale_name', 'description'], dtype='object')
Columns in oil_df: Index(['date', 'dcoilwtico'], dtype='object')
Columns in stores_df: Index(['store_nbr', 'city', 'state', 'type', 'cluster'], dtype='object')


In [ ]:
train_df = train_df.merge(stores_df, on='store_nbr', how='left')
test_df = test_df.merge(stores_df, on='store_nbr', how='left')
print("Columns in train_df:", train_df.columns)
print("Columns in test_df:", test_df.columns)

In [ ]:
train_df = train_df.merge(oil_df, on='date', how='left')
test_df = test_df.merge(oil_df, on='date', how='left')
print("Columns in train_df:", train_df.columns)
print("Columns in test_df:", test_df.columns)

In [ ]:
#transferred TRUE/FALSE means if the holiday was transferred that year or not
train_df = train_df.merge(holidays_df[['date', 'type']], on='date', how='left')
test_df = test_df.merge(holidays_df[['date', 'type']], on='date', how='left')
print("Columns in train_df:", train_df.columns)
print("Columns in test_df:", test_df.columns)

In [ ]:
print(train_df)
print(test_df)

In [ ]:
#rename type_x, type_y
train_df.rename(columns={'type_x': 'store_type', 'type_y': 'holiday_type'}, inplace=True)
test_df.rename(columns={'type_x': 'store_type', 'type_y': 'holiday_type'}, inplace=True)

# Check the column names after renaming
print(train_df)
print(test_df)

There is only 1 holiday date on the test set from the holidays_df dates, explaining why there are so many NaNs

In [ ]:
test_df['holiday_type'].fillna('No Holiday', inplace=True)

In [ ]:
#encode onpromotion, to represent if there was or wasn't a promotion
train_df['has_promotion'] = np.where(train_df['onpromotion'] > 0, 1, 0)
test_df['has_promotion'] = np.where(test_df['onpromotion'] > 0, 1, 0)

In [ ]:
print("Columns in train_df:", train_df.columns)
print("Columns in test_df:", test_df.columns)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

train_df['year_month'] = train_df['date'].dt.to_period('M')

#create a boxplot of sales by year_month
plt.figure(figsize=(12, 6))
sns.boxplot(x='year_month', y='sales', data=train_df)
plt.xticks(rotation=90)
plt.title('Boxplot of Sales by Month')
plt.show()

# EDA